In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [3]:
df = spark.read.parquet("data/unidos/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: float (nullable = true)
 |-- Ast: float (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: float (nullable = true)
 |-- PrgP: float (nullable = true)
 |-- PrgR: float (nullable = true)
 |-- Fls: float (nullable = true)
 |-- Off: float (nullable = true)
 |-- Crosses: float (nullable = true)
 |-- Recov: float (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: float (nullable = true)
 |-- Tack_Mid_3rd: float (nullable = true)
 |-- Tack_Att_3rd: float (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: float (nullable = true)
 |-- TklW: float (nullable = true)
 |-- Int: float (nullable = true)
 |--

In [4]:
df.count()

4969

In [11]:
df1 = spark.read.parquet("data/final/merge_jugadores.parquet")
df1.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Min: float (nullable = true)
 |-- Value: float (nullable = true)
 |-- performance_score: double (nullable = true)
 |-- penalty_score: double (nullable = true)
 |-- adjusted_score: double (nullable = true)
 |-- Evaluated_Position: string (nullable = true)



In [55]:
df.count()

4969

In [26]:
from pyspark.sql.functions import col



df_union = df1.toPandas()

df_union.to_csv("data/performance.csv", index=False)

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Definir una ventana por Posición, ordenada por adjusted_score descendente
window_spec = Window.partitionBy("Pos").orderBy(col("adjusted_score").desc())
df_2024_2025 = df1.filter(col("Season") == "2024-2025")
# Aplicar ranking
df_ranked = df1.withColumn("rank", rank().over(window_spec))

# Filtrar los mejores (rank = 1) por posición
df_top_by_position = df_ranked.filter(col("rank") == 1)

# Mostrar resultados
df_top_by_position.select(
    "Pos", "Player", "Squad", "Season", "adjusted_score", "performance_score", "penalty_score", "Value"
).orderBy("Pos").show(truncate=False)


NameError: name 'col' is not defined

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, trim, round, when
from pyspark.sql.types import FloatType
import builtins
position_metrics = {
    
    "Goalkeeper": {"Recov": 0.3, "Clearences": 0.2, "Pass_cmp%": 0.3, "GA": 0.3, "SoTA": 0.4, "Save%": 0.6, "CS%": 0.3, "Effective_Penalty_Saves": 0.2,
        "PSxG/SoT": 0.6, "PSxG": 0.6, "PSxG+/-": 0.5, "Launch%": 0.2, "Effective_Cross_stop": 0.5, "#OPA": 0.4
    },
    "Centre-Back": {
        "Int": 0.4, "Blocks": 0.3, "Clearences": 0.5, "Aerialwon%": 0.6,
        "Touch_Def_3rd": 0.2, "Tack_Def_3rd": 0.4, "PrgC": 0.1, "PrgP": 0.2, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.2, "Effective_Pass_Long": 0.3,
        "Effective_Tkl": 0.5, "Block_Shots": 0.4, "Effective_Tackles_vsDrib": 0.3,
        "PrgDist": 0.1
    },
    "Left-Back":{ 
        "PrgC": 0.3, "PrgP": 0.2, "Crosses": 0.4, "Recov": 0.4, "Tack_Def_3rd": 0.4, "Effective_Tkl": 0.5, "Int": 0.5,
        "Block_Shots": 0.4, "Block_Pass": 0.4, "Clearences": 0.5, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.1, "xA": 0.2,
        "A-xAG": 0.2, "CrsPA": 0.3, "Touch_Def_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.2,
        "PrgDist": 0.4, "Carries_Att_3rd": 0.2, "Touch_Live": 0.3, "Effective_Tackles_vsDrib": 0.5
    },
    "Right-Back": {
        "PrgC": 0.3, "PrgP": 0.2, "Crosses": 0.4, "Recov": 0.4, "Tack_Def_3rd": 0.4, "Effective_Tkl": 0.5, "Int": 0.5,
        "Block_Shots": 0.4, "Block_Pass": 0.4, "Clearences": 0.5, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.1, "xA": 0.2,
        "A-xAG": 0.2, "CrsPA": 0.3, "Touch_Def_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.2,
        "PrgDist": 0.4, "Carries_Att_3rd": 0.2, "Touch_Live": 0.3, "Effective_Tackles_vsDrib": 0.5
    },
    "Defensive Midfield": {
        "Int": 0.4, "Blocks": 0.5, "Clearences": 0.3, "Aerialwon%": 0.3, "Touch_Def_3rd": 0.2,
        "Touch_Mid_3rd": 0.4, "Tack_Def_3rd": 0.4, "Effective_Pass_Short": 0.6,
        "Effective_Pass_Medium": 0.5, "Effective_Pass_Long": 0.5, "PrgC": 0.2, "PrgP": 0.3,
        "Effective_Tkl": 0.5, "Block_Shots": 0.3,"PrgDist": 0.2, "Recov": 0.4,
        "Tack_Mid_3rd": 0.3, "Touch_Live": 0.4
   } ,
    "Central Midfield": {
        "Ast": 0.2, "PrgC": 0.3, "PrgP": 0.4, "Recov": 0.3, "Aerialwon%": 0.2, "Tack_Def_3rd": 0.3, "Tack_Mid_3rd": 0.4,
        "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.4, "Int": 0.4, "Block_Pass": 0.3,"Effective_Pass_Short": 0.6,
        "Effective_Pass_Medium": 0.6, "Effective_Pass_Long": 0.5, "xAG":0.2, "xA": 0.2, "A-xAG": 0.2, "Pass_cmp_Att_3rd": 0.3, "PPA": 0.4,
        "Touch_Def_3rd": 0.3, "Touch_Mid_3rd": 0.4, "Touch_Att_3rd": 0.3, "Effective_Tackles_vsDrib": 0.2,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Touch_Live": 0.2, "Effective_Drib": 0.2
    },
    "Attacking Midfield": {
        "Ast": 0.3, "PrgC": 0.3, "PrgP": 0.4, "PrgR": 0.4, "Tack_Att_3rd": 0.3, "Tack_Mid_3rd": 0.2, "Block_Pass": 0.2,
        "Effective_Pass_Short": 0.5, "Effective_Pass_Medium": 0.5, "Effective_Pass_Long": 0.3, "xAG": 0.5, "xA": 0.4, "A-xAG":0.3, "Pass_cmp_Att_3rd": 0.5, "PPA": 0.4,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Touch_Att_Pen": 0.2, "Effective_Drib": 0.3,
        "Effective_Tackles_vsDrib": 0.1, "PrgDist": 0.2, "Carries_Att_3rd": 0.3, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.3, "G/Sh": 0.2, "Touch_Live": 0.2
    },
    "Right Midfield": {
        "Gls": 0.1, "Ast": 0.2, "PrgC": 0.4, "PrgP": 0.3, "PrgR": 0.3, "Crosses": 0.4, "Tack_Mid_3rd": 0.3, "Tack_Att_3rd": 0.3,
        "Effective_Tkl": 0.2, "Block_Pass": 0.2, "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.4, "Effective_Pass_Long": 0.2, "xAG": 0.3, "xA": 0.3, "A-xAG": 0.3, "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "CrsPA": 0.5,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.5, "Effective_Tackles_vsDrib": 0.2, "PrgDist": 0.4, "Carries_Att_3rd": 0.5, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.2, "G/Sh": 0.2
   } ,
    "Left Midfield": {
        "Gls": 0.1, "Ast": 0.2, "PrgC": 0.4, "PrgP": 0.3, "PrgR": 0.3, "Crosses": 0.4, "Tack_Mid_3rd": 0.3, "Tack_Att_3rd": 0.3,
        "Effective_Tkl": 0.2, "Block_Pass": 0.2, "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.4, "Effective_Pass_Long": 0.2, "xAG": 0.3, "xA": 0.3, "A-xAG": 0.4, "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "CrsPA": 0.5,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.5, "Effective_Tackles_vsDrib": 0.2, "PrgDist": 0.4, "Carries_Att_3rd": 0.5, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.2, "G/Sh": 0.2
    },
    "Left Winger": {
        "Gls": 0.2, "Ast": 0.3, "PrgC": 0.5, "PrgP": 0.4, "PrgR": 0.4, "Crosses": 0.3, "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.2,
        "Block_Pass": 0.1, "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "PPA": 0.5, "CrsPA": 0.6, "Touch_Att_3rd": 0.5,
        "Touch_Att_Pen": 0.5, "Effective_Drib": 0.6, "Effective_Tackles_vsDrib": 0.1,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Carries_Att_Pen": 0.5, "xAG": 0.3, "xA": 0.3,
        "Sh": 0.4, "SoT": 0.3, "G/Sh": 0.3, "xG": 0.5, "npxG": 0.3, "G-xG": 0.3
    },
    "Right Winger": {
        "Gls": 0.2, "Ast": 0.3, "PrgC": 0.5, "PrgP": 0.4, "PrgR": 0.4, "Crosses": 0.3, "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.2,
        "Block_Pass": 0.1, "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "PPA": 0.5, "CrsPA": 0.6, "Touch_Att_3rd": 0.5,
        "Touch_Att_Pen": 0.5, "Effective_Drib": 0.6, "Effective_Tackles_vsDrib": 0.1,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Carries_Att_Pen": 0.5, "xAG": 0.3, "xA": 0.3,
        "Sh": 0.4, "SoT": 0.3, "G/Sh": 0.3, "xG": 0.5, "npxG": 0.3, "G-xG": 0.3
    },
    "Second Striker": {
        "Gls": 0.3, "Ast": 0.3, "PrgC": 0.4, "PrgP": 0.5, "PrgR": 0.6, "Tack_Att_3rd": 0.4, "Block_Pass": 0.1,
        "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "xAG": 0.5, "xA": 0.4,
        "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.6, "Touch_Att_Pen": 0.5,
        "Effective_Drib":0.4, "Effective_Tackles_vsDrib": 0.1, "PrgDist": 0.3,
        "Carries_Att_3rd": 0.3, "Carries_Att_Pen": 0.4,
        "Sh": 0.3, "SoT": 0.6, "G/Sh": 0.5, "xG": 0.6, "npxG": 0.4, "G-xG": 0.5, "Touch_Live": 0.2, "Effective_Tkl": 0.1
    },
    "Centre-Forward": {
        "Gls":0.3, "Ast":0.2, "PrgR": 0.3, "Aerialwon%": 0.5, "Tack_Att_3rd": 0.5, "Effective_Tkl": 0.4, "Block_Pass": 0.2,
        "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.2, "Effective_Pass_Long": 0.1, "xAG": 0.4,
        "xA": 0.4, "Pass_cmp_Att_3rd": 0.4, "Touch_Att_3rd": 0.5, "Touch_Mid_3rd": 0.2, "Touch_Att_Pen": 0.6,
        "Carries_Att_Pen": 0.5, "Effective_Drib": 0.4, 
        "Sh": 0.5, "SoT": 0.6, "G/Sh": 0.5, "xG": 0.7, "npxG": 0.5, "G-xG": 0.4
    }
}
def normalize_weights_by_position(position_metrics):
    normalized = {}
    for position, weights in position_metrics.items():
        total = sum(weights.values())
        if total == 0:
            raise ValueError(f"La posición '{position}' tiene una suma de pesos igual a cero.")
        normalized[position] = [
            (metric, builtins.round(weight / total, 4)) for metric, weight in weights.items()
        ]
    return normalized


In [85]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import DenseVector
from functools import reduce
from pyspark.ml.functions import vector_to_array


from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql.functions import col, lit
from functools import reduce

def calcular_performance_score_con_pesos(df, position_metrics_normalized):
    def procesar_posicion_x_temporada(df, pos, metric_weight_tuples):
        available_columns = df.columns
        seasons = [row["Season"] for row in df.select("Season").distinct().collect()]
        df_pos_season_list = []

        for season in seasons:
            df_season = df.filter((col("Pos") == pos) & (col("Season") == season))

            metrics = [m for m, _ in metric_weight_tuples if m in available_columns]
            weights = [w for m, w in metric_weight_tuples if m in available_columns]
            metric_weight_tuples_filtered = list(zip(metrics, weights))

            if not metric_weight_tuples_filtered:
                continue

            df_season = df_season.fillna(0, metrics)

            assembler = VectorAssembler(inputCols=metrics, outputCol="features_vec")
            df_season = assembler.transform(df_season)

            scaler = MinMaxScaler(inputCol="features_vec", outputCol="scaled_features")
            scaler_model = scaler.fit(df_season)
            df_season = scaler_model.transform(df_season)

            # ✅ convertir el vector a array usando librería oficial de PySpark ML
            df_season = df_season.withColumn("scaled_array", vector_to_array("scaled_features"))

            # calcular columnas ponderadas
            weighted_cols = []
            for i, (metric, weight) in enumerate(metric_weight_tuples_filtered):
                weighted_col_name = f"{metric}_weighted"
                df_season = df_season.withColumn(weighted_col_name, col("scaled_array")[i] * lit(weight))
                weighted_cols.append(col(weighted_col_name))

            if weighted_cols:
                df_season = df_season.withColumn("performance_score", sum(weighted_cols))
            else:
                df_season = df_season.withColumn("performance_score", lit(0.0))

            # eliminar columnas temporales
            cols_to_drop = ["features_vec", "scaled_features", "scaled_array"] + \
                           [f"{metric}_weighted" for metric, _ in metric_weight_tuples_filtered]
            df_season = df_season.drop(*cols_to_drop)

            df_season = df_season.withColumn("Evaluated_Position", lit(pos))
            df_pos_season_list.append(df_season)

        return reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), df_pos_season_list) if df_pos_season_list else df.limit(0)

    scored_dfs = []
    for pos, metric_weight_tuples in position_metrics_normalized.items():
        if metric_weight_tuples:
            scored_df = procesar_posicion_x_temporada(df, pos, metric_weight_tuples)
            if scored_df.count() > 0:
                scored_dfs.append(scored_df)

    return reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), scored_dfs) if scored_dfs else df.limit(0)


print("normalizar pesos...")
position_metrics_normalized = normalize_weights_by_position(position_metrics)
print("calculando el performance score...")
df = calcular_performance_score_con_pesos(df, position_metrics_normalized)
    

normalizar pesos...
calculando el performance score...


In [77]:
df_clean = spark.createDataFrame(df.rdd, schema=df.schema)
df_clean.show()


Py4JJavaError: An error occurred while calling o40286.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 940.0 failed 1 times, most recent failure: Lost task 0.0 in stage 940.0 (TID 837) (LAPTOP-SNOCNEUP.mshome.net executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:200)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:110)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:123)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.$anonfun$

In [75]:
print(df.columns)


['Player', 'Squad', 'Season', 'Competition', 'Born', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'YellowC', 'RedC', 'PrgC', 'PrgP', 'PrgR', 'Fls', 'Off', 'Crosses', 'Recov', 'Aerialwon%', 'Tack_Def_3rd', 'Tack_Mid_3rd', 'Tack_Att_3rd', 'Tkl%', 'Tkl', 'TklW', 'Int', 'Blocks', 'Block_Shots', 'Block_Pass', 'Clearences', 'Err', 'Pass_cmp', 'Pass_cmp%', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'Pass_Long', 'Pass_cmp_Long%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Def_3rd', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen', 'Touch_Live', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen', 'fail_To_Gain_Control', 'Loss_Control_Tackle', 'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'Pos', 'Height', 'Value', 'GA', 'SoTA', 'Save%', 'CS%', 'PKatt', 'P_Save%', 'PSxG', 'PSxG/SoT', 'PSxG+/-', 'Launch%', 'Crosses_Opp', 'Crosses_stp%', '#OPA', 'Gls_per90', 'Ast_per90', 'PrgC_per90', 'PrgP_per90', 'Pr

In [74]:
df.show()

Py4JJavaError: An error occurred while calling o40125.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 938.0 failed 1 times, most recent failure: Lost task 0.0 in stage 938.0 (TID 835) (LAPTOP-SNOCNEUP.mshome.net executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:200)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:110)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:123)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.$anonfun$

In [4]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("performance_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1).select("Player", "Pos", "Squad", "Competition", "performance_score").show()


+--------------------+------------------+---------------+--------------+-------------------+
|              Player|               Pos|          Squad|   Competition|  performance_score|
+--------------------+------------------+---------------+--------------+-------------------+
|       florian wirtz|Attacking Midfield|     leverkusen|    bundesliga| 0.4698577307771978|
|               pedri|  Central Midfield|      barcelona|        laliga| 0.4287534923294778|
|          mario gila|       Centre-Back|          lazio|       serie-a|0.32671365296692373|
|     shumaira mheuka|    Centre-Forward|        chelsea|premier-league|             0.4063|
|      joshua kimmich|Defensive Midfield|  bayern munich|    bundesliga| 0.5666147573187089|
|           pau lopez|        Goalkeeper|         girona|        laliga|0.47719619945274167|
|    marcus tavernier|     Left Midfield|    bournemouth|premier-league| 0.5959664719230389|
|         jeremy doku|       Left Winger|manchester city|premier-leagu

In [14]:
df1.columns

['Player',
 'Squad',
 'Season',
 'Competition',
 'Pos',
 'Min',
 'Value',
 'performance_score',
 'penalty_score',
 'adjusted_score',
 'Evaluated_Position']

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, position, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("performance_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("pos").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") < 11) \
                        .filter(col("pos") == position)\
                        .filter(col("Min") > 600)\
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1, "Centre-Forward").select("Player", "Pos", "Squad", "Competition", "performance_score", "penalty_score", "adjusted_score").show()

+-------------------+--------------+---------------+-----------+-------------------+--------------------+-------------------+
|             Player|           Pos|          Squad|Competition|  performance_score|       penalty_score|     adjusted_score|
+-------------------+--------------+---------------+-----------+-------------------+--------------------+-------------------+
|      kylian mbappe|Centre-Forward|    real madrid|     laliga| 0.2146943242814554|              0.0928|0.12189432428145541|
|benedict hollerbach|Centre-Forward|   union berlin| bundesliga|0.15549765424037695|              0.1348|0.02069765424037695|
|  antoine griezmann|Centre-Forward|atlético madrid|     laliga|0.15420179685048288|0.045700000000000005|0.10850179685048288|
|         harry kane|Centre-Forward|  bayern munich| bundesliga| 0.1496455294043452|              0.0475| 0.1021455294043452|
|     mohamed amoura|Centre-Forward|      wolfsburg| bundesliga| 0.1466798520285576| 0.07310000000000001|0.07357985202

In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, position, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("adjusted_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("pos").orderBy(col("adjusted_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") < 11) \
                        .filter(col("pos") == position)\
                        .filter(col("Min") > 600)\
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1, "Centre-Forward").select("Player", "Pos", "Squad", "Competition", "performance_score", "penalty_score", "adjusted_score").show()

+-----------------+--------------+---------------+-----------+-------------------+--------------------+-------------------+
|           Player|           Pos|          Squad|Competition|  performance_score|       penalty_score|     adjusted_score|
+-----------------+--------------+---------------+-----------+-------------------+--------------------+-------------------+
|    kylian mbappe|Centre-Forward|    real madrid|     laliga| 0.2146943242814554|              0.0928|0.12189432428145541|
|antoine griezmann|Centre-Forward|atlético madrid|     laliga|0.15420179685048288|0.045700000000000005|0.10850179685048288|
|       harry kane|Centre-Forward|  bayern munich| bundesliga| 0.1496455294043452|              0.0475| 0.1021455294043452|
|       iago aspas|Centre-Forward|     celta vigo|     laliga|0.14606093587227076|              0.0522|0.09386093587227076|
|   marvin ducksch|Centre-Forward|  werder bremen| bundesliga|0.11710087121811101|              0.0378|0.07930087121811101|
|    ger

In [11]:
df1 = spark.read.parquet("data/final/merge_jugadores.parquet")
df1.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- performance_score: double (nullable = true)
 |-- penalty_score: double (nullable = true)
 |-- adjusted_score: double (nullable = true)
 |-- Evaluated_Position: string (nullable = true)



In [27]:
df1.filter(col("Player") == "thibaut courtois").show()

+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+------------------+------------------+
|          Player|      Squad|   Season|Competition|       Pos|   Min|Value| performance_score|penalty_score|    adjusted_score|Evaluated_Position|
+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+------------------+------------------+
|thibaut courtois|real madrid|2023-2024|     laliga|Goalkeeper| 332.0|2.8E7|0.3738839432952276|          0.0|0.3738839432952276|        Goalkeeper|
|thibaut courtois|real madrid|2024-2025|     laliga|Goalkeeper|2700.0|2.0E7|0.3363258408796188|       0.0015|0.3348258408796188|        Goalkeeper|
|thibaut courtois|real madrid|2022-2023|     laliga|Goalkeeper|2790.0|4.5E7|0.4346668970958383|       0.0021|0.4325668970958383|        Goalkeeper|
+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+---

In [28]:
from pyspark.sql.functions import col, desc

# Filtro mínimo de minutos por posición (ajustable)

def top_10_por_posicion_temporada(df, posicion, temporada, variable_score):
    min_minutos = 600

    return (
        df.filter((col("Season") == temporada) & (col("Min") >= min_minutos))
          .orderBy(desc(variable_score))
          .limit(10)
    )
top_10_por_posicion_temporada(df1, "Goalkeeper", "2024-2025", "adjusted_score").show()

+-----------------+---------------+---------+--------------+------------------+------+---------+-------------------+--------------------+-------------------+------------------+
|           Player|          Squad|   Season|   Competition|               Pos|   Min|    Value|  performance_score|       penalty_score|     adjusted_score|Evaluated_Position|
+-----------------+---------------+---------+--------------+------------------+------+---------+-------------------+--------------------+-------------------+------------------+
|    juan cuadrado|       atalanta|2024-2025|       serie-a|    Right Midfield| 832.0|1000000.0| 0.6739622561520988|0.046400000000000004| 0.6275622561520988|    Right Midfield|
|   joshua kimmich|  bayern munich|2024-2025|    bundesliga|Defensive Midfield|2847.0|    4.5E7| 0.5666147573187089|0.020200000000000003| 0.5464147573187089|Defensive Midfield|
| marcus tavernier|    bournemouth|2024-2025|premier-league|     Left Midfield|1939.0|    1.8E7| 0.5959664719230389

In [40]:
from pyspark.sql.functions import col, desc

# Filtro mínimo de minutos por posición (ajustable)

def top_10_por_posicion_temporada(df, posicion, temporada, variable_score):
    min_minutos = 600

    return (
        df.filter((col("Pos") == posicion) & (col("Season") == temporada) & (col("Min") >= min_minutos))
          .orderBy(desc(variable_score))
          .limit(10)
    )
top_10_por_posicion_temporada(df1, "Centre-Forward", "2024-2025", "adjusted_score").show()


+-----------------+---------------+---------+--------------+--------------+------+---------+-------------------+--------------------+-------------------+------------------+
|           Player|          Squad|   Season|   Competition|           Pos|   Min|    Value|  performance_score|       penalty_score|     adjusted_score|Evaluated_Position|
+-----------------+---------------+---------+--------------+--------------+------+---------+-------------------+--------------------+-------------------+------------------+
|    kylian mbappe|    real madrid|2024-2025|        laliga|Centre-Forward|2907.0|    1.8E8| 0.2146943242814554|              0.0928|0.12189432428145541|    Centre-Forward|
|antoine griezmann|atlético madrid|2024-2025|        laliga|Centre-Forward|2469.0|    1.8E7|0.15420179685048288|0.045700000000000005|0.10850179685048288|    Centre-Forward|
|       harry kane|  bayern munich|2024-2025|    bundesliga|Centre-Forward|2381.0|    7.5E7| 0.1496455294043452|              0.0475| 0

In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("performance_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1).select("Player", "Pos", "Squad", "Competition", "adjusted_score", "performance_score", "penalty_score").show()

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, min, max, when

def ajustar_penalty_score(df):
    # Ventana por posición y temporada
    w = Window.partitionBy("Evaluated_Position", "Season")

    # Calcular mínimo y máximo por grupo
    df = df.withColumn("penalty_min", min("penalty_score").over(w)) \
           .withColumn("penalty_max", max("penalty_score").over(w))

    # Normalizar penalty_score a [0, 1]
    df = df.withColumn(
        "penalty_score_scaled",
        when(
            col("penalty_max") != col("penalty_min"),
            (col("penalty_score") - col("penalty_min")) / (col("penalty_max") - col("penalty_min"))
        ).otherwise(0.0)
    )

    # Ajustar performance_score
    df = df.withColumn(
        "adjusted_score",
        col("performance_score") - col("penalty_score_scaled")
    )

    # Limpiar columnas auxiliares si no las necesitas
    df = df.drop("penalty_min", "penalty_max")

    return df

df2 = ajustar_penalty_score(df1)

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("adjusted_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 2) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df2).select("Player", "Pos", "Squad", "Competition", "adjusted_score", "performance_score", "penalty_score").show()

+-----------------+------------------+--------------+--------------+--------------------+-------------------+------------------+
|           Player|               Pos|         Squad|   Competition|      adjusted_score|  performance_score|     penalty_score|
+-----------------+------------------+--------------+--------------+--------------------+-------------------+------------------+
|  bruno fernandes|Attacking Midfield|manchester utd|premier-league| 0.25164164559369295| 0.4600860900381374|0.9380000000000001|
|      jakub moder|  Central Midfield|      brighton|premier-league|  0.3638061210290161| 0.3638061210290161|               0.0|
|      daley blind|       Centre-Back|        girona|        laliga|  0.2961812019322885| 0.3261812019322885|             0.405|
|    tammy abraham|    Centre-Forward|          roma|       serie-a| -0.7395062725298683|0.26049372747013166|              18.0|
| manuel locatelli|Defensive Midfield|      juventus|       serie-a| 0.34723869069005425| 0.46543